In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *

Setting environment for Precision WorkStation
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_sorted_filenames.txt" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_sorted_filenames.txt" "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_sorted_filenames.txt"


No vtk
No vtk

- After receiving data of a new stack, put images and macros in corresponding folder.
- Add the stack name to proper variables in `metadata.txt`.

In [2]:
stack = 'MD585'
# stack = 'ChatCryoJane201710'

In [3]:
# tb_fmt = 'png'
tb_fmt = 'tif'

- Use GUI, quality check and sort images.
- Upload `sorted_filenames.txt` to S3

In [9]:
# transfer_data_synced(os.path.join('CSHL_data', stack), 
#                      from_hostname='s3raw', to_hostname='ec2', is_dir=True, include_only='*.'+tb_fmt)

In [7]:
transfer_data_synced(DataManager.get_sorted_filenames_filename(stack=stack), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_sorted_filenames.txt" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585"
aws s3 cp "s3:///media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_sorted_filenames.txt" "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_sorted_filenames.txt"


# Crop

In [146]:
# stack = 'MD661'
# download_from_s3(DataManager.get_cropbox_filename(stack), redownload=True)

rm -rf "/shared/CSHL_data_processed/MD661/MD661_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropbox.txt" && mkdir -p "/shared/CSHL_data_processed/MD661"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD661/MD661_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropbox.txt" "/shared/CSHL_data_processed/MD661/MD661_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropbox.txt"


In [7]:
xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox_thalamus(stack=stack)
w = xmax + 1 - xmin
h = ymax + 1 - ymin
x = xmin
y = ymin

print x,y,w,h

rm -rf "/shared/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox_thalamus.txt" && mkdir -p "/shared/CSHL_data_processed/MD585"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox_thalamus.txt" "/shared/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox_thalamus.txt"
433 203 277 219


# Crop Thumbnail

In [9]:
input_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=1, resol='thumbnail')
download_from_s3(input_dir, is_dir=True, local_root=DATA_ROOTDIR)

# prep=3 for cropping of thalamus
output_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=3, resol='thumbnail')

execute_command('rm -r ' + output_dir);
execute_command('mkdir -p ' + output_dir);

rm -rf "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail" && mkdir -p "/shared/CSHL_data_processed/MD585"
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_prep1_thumbnail" "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail"
rm -r /shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail
mkdir -p /shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail


In [10]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [11]:
t = time.time()
sys.stderr.write('cropping thumbnail...')

for fn in valid_filenames:

    execute_command('convert \"%(input_fp)s\" -crop %(w)dx%(h)d+%(x)d+%(y)d  \"%(output_fp)s\"' % \
        {'stack': stack,
        'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=1, fn=fn, resol='thumbnail'),
        'output_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=3, fn=fn, resol='thumbnail'),
        'w':xmax+1-xmin, 'h':ymax+1-ymin, 'x':xmin, 'y':ymin})

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70s (aws)

cropping thumbnail...

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N3-2015.07.16-19.48.34_MD585_1_0007_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N3-2015.07.16-19.48.34_MD585_1_0007_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC3-2015.07.16-15.26.44_MD585_1_0007_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC3-2015.07.16-15.26.44_MD585_1_0007_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N3-2015.07.16-19.48.34_MD585_2_0008_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N3-2015.07.16-19.48.34_MD585_2_0008_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC3-2015.07.16-15.26.44_MD585_2_0008_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/M

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC10-2015.07.16-14.55.58_MD585_1_0028_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC10-2015.07.16-14.55.58_MD585_1_0028_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N10-2015.07.16-20.11.10_MD585_2_0029_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N10-2015.07.16-20.11.10_MD585_2_0029_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC10-2015.07.16-14.55.58_MD585_2_0029_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC10-2015.07.16-14.55.58_MD585_2_0029_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N10-2015.07.16-20.11.10_MD585_3_0030_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC15-2015.07.16-15.14.52_MD585_1_0043_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC15-2015.07.16-15.14.52_MD585_1_0043_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N15-2016.03.07-17.56.36_MD585_2_0044_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N15-2016.03.07-17.56.36_MD585_2_0044_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC15-2015.07.30-16.05.22_MD585_2_0044_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC15-2015.07.30-16.05.22_MD585_2_0044_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N15-2016.03.07-17.56.36_MD585_3_0045_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC20-2015.07.16-15.43.53_MD585_2_0059_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC20-2015.07.16-15.43.53_MD585_2_0059_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N20-2015.07.16-20.47.19_MD585_3_0060_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N20-2015.07.16-20.47.19_MD585_3_0060_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC20-2015.07.16-15.43.53_MD585_3_0060_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC20-2015.07.16-15.43.53_MD585_3_0060_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N21-2015.07.18-07.06.33_MD585_1_0061_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

...

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC60-2015.08.19-15.25.07_MD585_3_0180_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC60-2015.08.19-15.25.07_MD585_3_0180_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N61-2015.07.16-23.52.50_MD585_1_0181_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N61-2015.07.16-23.52.50_MD585_1_0181_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC61-2015.07.16-18.50.40_MD585_3_0183_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC61-2015.07.16-18.50.40_MD585_3_0183_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N61-2015.07.16-23.52.50_MD585_2_0182_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC65-2015.07.16-19.07.14_MD585_3_0195_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC65-2015.07.16-19.07.14_MD585_3_0195_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N66-2015.07.17-00.13.23_MD585_1_0196_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N66-2015.07.17-00.13.23_MD585_1_0196_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC66-2015.07.16-19.11.12_MD585_1_0196_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC66-2015.07.16-19.11.12_MD585_1_0196_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N66-2015.07.17-00.13.23_MD585_2_0197_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC70-2015.07.16-19.26.09_MD585_1_0208_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC70-2015.07.16-19.26.09_MD585_1_0208_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N71-2015.07.17-00.31.56_MD585_1_0211_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N71-2015.07.17-00.31.56_MD585_1_0211_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC71-2015.07.16-19.30.50_MD585_1_0211_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC71-2015.07.16-19.30.50_MD585_1_0211_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N71-2015.07.17-00.31.56_MD585_2_0212_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_pr

convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N76-2015.07.17-00.46.27_MD585_1_0226_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N76-2015.07.17-00.46.27_MD585_1_0226_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC76-2015.07.16-19.46.08_MD585_1_0226_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-IHC76-2015.07.16-19.46.08_MD585_1_0226_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-N76-2015.07.17-00.46.27_MD585_2_0227_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail/MD585-N76-2015.07.17-00.46.27_MD585_2_0227_prep3_thumbnail.tif"
convert "/shared/CSHL_data_processed/MD585/MD585_prep1_thumbnail/MD585-IHC76-2015.07.16-19.46.08_MD585_2_0227_prep1_thumbnail.tif" -crop 277x219+433+203  "/shared/CSHL_data_proc

done in 37.729704 seconds


In [12]:
upload_to_s3(output_dir, is_dir=True, local_root=DATA_ROOTDIR)

aws s3 cp --recursive "/shared/CSHL_data_processed/MD585/MD585_prep3_thumbnail" "s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_prep3_thumbnail"


# Expand lossless JP2 (use /shared)

In [79]:
stack = 'MD661'

In [ ]:
transfer_data_synced(os.path.join('CSHL_data', stack),
                    from_hostname='s3raw',
                    to_hostname='ec2',
                    is_dir=True,
                    include_only='*_lossless.jp2')

aws s3 cp --recursive "s3://mousebrainatlas-rawdata/CSHL_data/MD661" "/shared/CSHL_data/MD661" --exclude "*" --include "*_lossless.jp2"


In [13]:
DataManager.get_image_dir_v2(stack=stack, prep_id=1, resol='lossless')

'/shared/CSHL_data_processed/MD585/MD585_prep1_lossless'

In [88]:
output_dir = create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=1, resol='lossless'))

In [ ]:
t = time.time()
sys.stderr.write('expanding...')

in_fps = [fn + '_lossless.jp2' for fn in metadata_cache['valid_filenames'][stack]]
out_fps = [DataManager.get_image_filepath_v2(stack=stack, prep_id=1, resol='lossless', fn=fn) 
           for fn in metadata_cache['valid_filenames'][stack]]

run_distributed('export LD_LIBRARY_PATH=/home/ubuntu/KDU79_Demo_Apps_for_Linux-x86-64_170108:$LD_LIBRARY_PATH; %(kdu_bin)s -i \"%%(in_fp)s\" -o \"%%(out_fp)s\"' % \
                {'kdu_bin': KDU_EXPAND_BIN},
                kwargs_list={'in_fp': in_fps, 'out_fp': out_fps},
                argument_type='single',
               cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 seconds.

In [ ]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_lossless_tif'),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True) # 6000s

# Warping and cropping lossless (use /shared)

In [24]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_lossless_tif'),
                    from_hostname='s3',
                    to_hostname='ec2',
                    is_dir=True)
# 3000 seconds

rm -rf /shared/CSHL_data_processed/MD585/MD585_lossless_tif && mkdir -p /shared/CSHL_data_processed/MD585
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_lossless_tif /shared/CSHL_data_processed/MD585/MD585_lossless_tif


Child returned 0


3079.07781506


Child returned 0
3079.08 seconds.


In [37]:
# tf_filepath = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % {'stack':stack})
# tfs = pickle.load(open(tf_filepath, 'r'))
# Note that the index from trasform pickle file starts at 0, BUT the .._renamed folder index starts at 1.#

tfs = DataManager.load_transforms(stack=stack)

lossless_tif_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_tif')
lossless_aligned_cropped_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped')

script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')

In [45]:
! rm -r {lossless_aligned_cropped_dir}

In [43]:
if stack in all_nissl_stacks:
    pad_bg_color = 'white'
else:
    pad_bg_color = 'auto'

In [ ]:
t = time.time()
sys.stderr.write('warping and cropping lossless...')

# wait_num_nodes(16)
                   
if pad_bg_color == 'auto':
    # If alternating, then black padding for F sections, white padding for N sections.
    run_distributed(command='%(script_path)s %(stack)s %(lossless_tif_dir)s %(lossless_aligned_cropped_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'lossless_tif_dir': lossless_tif_dir,
                    'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h},
                    kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                'filename': fn + '_lossless.tif',
                                'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif',
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],
#                                  for fn in valid_filenames[first_idx_among_valid:first_idx_among_valid+16]],
                    argument_type='single',
                   cluster_size=16,
                   jobs_per_node=4)
else:
    run_distributed(command='%(script_path)s %(stack)s %(lossless_tif_dir)s %(lossless_aligned_cropped_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'lossless_tif_dir': lossless_tif_dir,
                    'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                'filename': fn + '_lossless.tif',
                                'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif'}
                                for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],
#                                  for fn in valid_filenames[first_idx_among_valid:first_idx_among_valid+16]],
                    argument_type='single',
                   cluster_size=16,
                   jobs_per_node=4)

wait_qsub_complete()
    
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 4140 seconds (AWS)

In [24]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped'),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True) 
# 512 seconds

aws s3 cp --recursive /shared/CSHL_data_processed/MD658/MD658_lossless_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_lossless_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropped


Child returned 0
1064.17 seconds.


# Warp and crop lossless CryoJane

In [4]:
# version = 'Ntb'
version = 'Chat'

if stack in all_nissl_stacks:
    pad_bg_color = 'white'
elif stack == 'ChatCryoJane201710':
    pad_bg_color = 'black'
else:
    pad_bg_color = 'auto'

In [5]:
prep_id = 2

In [6]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [7]:
# RAW_DATA_DIR = os.path.join(DATA_ROOTDIR, 'CSHL_data')

In [8]:
# tf_filepath = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % {'stack':stack})
# tfs = pickle.load(open(tf_filepath, 'r'))
# Note that the index from trasform pickle file starts at 0, BUT the .._renamed folder index starts at 1.#

anchor_fn = DataManager.load_anchor_filename(stack=stack)
transforms_to_anchor = DataManager.load_transforms(stack=stack, downsample_factor=32, use_inverse=False, anchor_fn=anchor_fn)

input_dir = os.path.join(RAW_DATA_DIR, stack, stack + '_raw_' + version)
# output_dir = os.path.join(DATA_DIR, stack, stack + '_prep' + prep_id + _lossless_' + version)
output_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=prep_id, resol='lossless', version=version)

script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')

In [9]:
xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
w = xmax + 1 - xmin
h = ymax + 1 - ymin
x = xmin
y = ymin

print x,y,w,h

509 72 949 696


In [10]:
t = time.time()

if pad_bg_color == 'auto': # useful for alternatively stained stacks where bg varies depending on stain on each section
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                     'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    },
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': os.path.join(input_dir, \
                                                        fn + '_lossless_' + version + '.' + tb_fmt \
                                              if version is not None \
                                              else fn + '.' + tb_fmt),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=prep_id, version=version, resol='lossless'),
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in valid_filenames],
                    argument_type='single',
                   jobs_per_node=1,
                   local_only=True)
else:
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s lossless %(x)d %(y)d %(w)d %(h)d %(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                     'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': os.path.join(input_dir, \
                                                        fn + '_lossless_' + version + '.' + tb_fmt \
                                              if version is not None \
                                              else fn + '.' + tb_fmt),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=prep_id, version=version, resol='lossless'),
                                 }
                                for fn in valid_filenames],
                    argument_type='single',
                   jobs_per_node=1,
                   local_only=True)

# wait_qsub_complete()
    
print 'done in', time.time() - t, 'seconds' # 300 seconds.

rm -f ~/stderr_*; rm -f ~/stdout_*


Run locally.


done in 19019.3771439 seconds


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


# Expand and Crop together (regular, local, non-distributed)

In [4]:
stack = 'MD585'

In [5]:
tfs = DataManager.load_transforms(stack=stack, downsample_factor=32, use_inverse=False)
# xmin, xmax, ymin, ymax = metadata_cache['cropbox'][stack]
xmin, xmax, ymin, ymax = DataManager.load_cropbox_thalamus(stack)[:4]
print xmin, xmax, ymin, ymax

433 709 203 421


In [6]:
raw_jp2_dir = DataManager.get_image_dir(stack=stack, resol='lossless', version='original_jp2')
lossless_tif_dir = DataManager.get_image_dir_v2(stack=stack, resol='lossless', prep_id=0)
lossless_aligned_cropped_dir = DataManager.get_image_dir_v2(stack=stack, resol='lossless', prep_id=3)

In [18]:
# Linux does not support commands that are too long, so cannot stack too many filenames in the command.
for i in range(0, len(metadata_cache['valid_filenames'][stack]), 200):
    
    t = time.time()
    sys.stderr.write('expanding...')

    warp_crop_script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')

    run_distributed('aws s3 cp \"s3://mousebrainatlas-rawdata/CSHL_data/%(stack)s/%%(fn)s_lossless.jp2\" \"%(raw_jp2_dir)s\"/ && \
    mkdir -p \"%(lossless_tif_dir)s\" && \
    LD_LIBRARY_PATH=\"%(kdu_dir)s\":$LD_LIBRARY_PATH %(kdu_bin)s -i \"%%(in_fp)s\" -o \"%%(lossless_tif_fp)s\" && \
    mkdir -p \"%(lossless_aligned_cropped_dir)s\" && \
    %(script_path)s %(stack)s \"%%(lossless_tif_fp)s\" \"%%(lossless_aligned_cropped_fp)s\" %%(transform)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s && \
    aws s3 cp \"%%(lossless_aligned_cropped_fp)s\" \"s3://mousebrainatlas-data/%(s3_dest_dir)s/\" && \
    rm \"%%(lossless_tif_fp)s\" && rm \"%%(in_fp)s\"' % \
                    {'stack': stack,
                    'kdu_bin': KDU_EXPAND_BIN,
                     'kdu_dir': os.path.dirname(KDU_EXPAND_BIN),
                    'raw_jp2_dir': raw_jp2_dir,
                    'script_path': warp_crop_script_fp,
                    'lossless_tif_dir': lossless_tif_dir,
                    'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                     's3_dest_dir': relative_to_local(lossless_aligned_cropped_dir, local_root='/media/yuncong/BstemAtlasData/'),
                    'x': xmin,
                    'y': ymin,
                    'w': xmax + 1 - xmin,
                    'h': ymax + 1 - ymin},
                    kwargs_list=[{'fn': fn, 
                                  'in_fp': os.path.join(raw_jp2_dir, fn + '_lossless.jp2'),
                                  'transform': ','.join(map(str, tfs[fn].flatten())),
                                    'lossless_tif_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, resol='lossless', prep_id=0),
                                    'lossless_aligned_cropped_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, resol='lossless', prep_id=3),
                                    'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                 for fn in metadata_cache['valid_filenames'][stack][i:i+200]],
                    argument_type='single',
                   local_only=True)

    # wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # 16 4.2xlarge, whole stack,4222 seconds.
    # 4 4.xlarge nodes, whole stack, 10280 seconds.

expanding...Run locally.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
done in 18231.760643 seconds
expanding...Run locally.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
done in 4772.454183 seconds


# Expand and Crop together (for AWS cluster, use /scratch)

In [6]:
stack = 'MD585'

In [7]:
tfs = DataManager.load_transforms(stack=stack, downsample_factor=32, use_inverse=False)
# xmin, xmax, ymin, ymax = metadata_cache['cropbox'][stack]
xmin, xmax, ymin, ymax = DataManager.load_cropbox_thalamus(stack)[:4]
print xmin, xmax, ymin, ymax

433 709 203 421


In [8]:
raw_jp2_dir = DataManager.get_image_dir(stack=stack, resol='lossless', version='original_jp2', raw_data_dir='/scratch/CSHL_data')
lossless_tif_dir = DataManager.get_image_dir_v2(stack=stack, resol='lossless', prep_id=0, data_dir='/scratch/CSHL_data_processed')
lossless_aligned_cropped_dir = DataManager.get_image_dir_v2(stack=stack, resol='lossless', prep_id=3, data_dir='/scratch/CSHL_data_processed')

In [9]:
run_distributed('rm -rf /scratch/*', argument_type='single', local_only=True)

rm -f ~/stderr_*; rm -f ~/stdout_*


Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [13]:
t = time.time()
sys.stderr.write('expanding...')

warp_crop_script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')

run_distributed('aws s3 cp \"s3://mousebrainatlas-rawdata/CSHL_data/%(stack)s/%%(fn)s_lossless.jp2\" \"%(raw_jp2_dir)s\"/ && \
mkdir -p \"%(lossless_tif_dir)s\" && \
LD_LIBRARY_PATH=/home/ubuntu/KDU79_Demo_Apps_for_Linux-x86-64_170108:$LD_LIBRARY_PATH %(kdu_bin)s -i \"%%(in_fp)s\" -o \"%%(lossless_tif_fp)s\" && \
mkdir -p \"%(lossless_aligned_cropped_dir)s\" && \
%(script_path)s %(stack)s \"%%(lossless_tif_fp)s\" \"%%(lossless_aligned_cropped_fp)s\" %%(transform)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s && \
aws s3 cp \"%%(lossless_aligned_cropped_fp)s\" \"s3://mousebrainatlas-data/%(s3_dest_dir)s/\" && \
rm \"%%(lossless_tif_fp)s\" && rm \"%%(in_fp)s\"' % \
                {'stack': stack,
                'kdu_bin': KDU_EXPAND_BIN,
                'raw_jp2_dir': raw_jp2_dir,
                'script_path': warp_crop_script_fp,
                'lossless_tif_dir': lossless_tif_dir,
                'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                 's3_dest_dir': relative_to_local(lossless_aligned_cropped_dir, local_root='/scratch'),
                'x': xmin,
                'y': ymin,
                'w': xmax + 1 - xmin,
                'h': ymax + 1 - ymin},
                kwargs_list=[{'fn': fn, 
                              'in_fp': os.path.join(raw_jp2_dir, fn + '_lossless.jp2'),
                              'transform': ','.join(map(str, tfs[fn].flatten())),
                                'lossless_tif_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, resol='lossless', prep_id=0, data_dir='/scratch/CSHL_data_processed'),
                                'lossless_aligned_cropped_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, resol='lossless', prep_id=3, data_dir='/scratch/CSHL_data_processed'),
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                             for fn in metadata_cache['valid_filenames'][stack][:200]],
                argument_type='single',
               local_only=True)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
# 16 4.2xlarge, whole stack,4222 seconds.
# 4 4.xlarge nodes, whole stack, 10280 seconds.

expanding...Run locally.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 25995.952769 seconds


In [14]:
! cat /tmp/stderr_0.log

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color: white
Background color:

# Convert images to nissl-like grayscale - for full nissl stacks

In [20]:
# for stack in all_nissl_stacks:
for stack in ['MD595']:
    
    download_from_s3(DataManager.get_sorted_filenames_filename(stack=stack))
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    download_from_s3(DataManager.get_anchor_filename_filename(stack=stack))

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)

    # Convert Nissl images to grayscale
    
    t = time.time()
    sys.stderr.write('Convert nissl images to gray...')

    script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
                                    },
                        argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 seconds.

Convert nissl images to gray...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1136.945903 seconds


# Convert images to nissl-like grayscale - for alternating nissl/neurotrace stacks

In [269]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD653', 'MD657']:
    
    download_from_s3(DataManager.get_sorted_filenames_filename(stack=stack))
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    download_from_s3(DataManager.get_anchor_filename_filename(stack=stack))

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)

    # Convert Nissl images to grayscale
    
    t = time.time()
    sys.stderr.write('Convert nissl images to gray...')

    script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 'stack': stack},
                        kwargs_list={'filenames': 
                                     [fn for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
                                      if fn.split('-')[1][0] == 'N']
                                    },
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 seconds.
    
    # Match intensity profile between Neurotrace Blue to Nissl

    t = time.time()
    sys.stderr.write('Match intensity profile between Neurotrace and Nissl...')

    filename_pairs = []
    l = valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
    for i, fn in enumerate(l):
        if l[i].split('-')[1][0] == 'F':
            for d in range(1, 99):
                if i+d < len(l) and l[i+d].split('-')[1][0] == 'N':
                    filename_pairs.append((l[i+d], l[i]))
                    break
                if i-d >= 0 and l[i-d].split('-')[1][0] == 'N':
                    filename_pairs.append((l[i-d], l[i]))
                    break
            
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'match_intensity_profile.py')
    
    run_distributed(command='%(script_path)s %(stack)s \'%%(filename_pairs)s\' ' % \
                    {'script_path': script_fp,
                    'stack': stack,
                    'filename_pairs': filename_pairs},
                    kwargs_list=dict(filename_pairs=filename_pairs),
                    argument_type='list2',
                   cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2500 seconds. 
    # TODO: One node is especially slow, investigate.
    
    # Convert Neurotrace images to grayscale
    
    t = time.time()
    sys.stderr.write('Convert neurotrace images to gray...')
    
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')
    
    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': [fn for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
                                      if fn.split('-')[1][0] == 'F']
                                    },
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1200 seconds.    

Convert nissl images to gray...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1098.395771 seconds
16 nodes requested, 16 nodes available...Continuing


Match intensity profile between Neurotrace and Nissl...

Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
Convert neurotrace images to gray...16 nodes requested, 16 nodes available...Continuing


 done in 2371.7916441 seconds


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1008.132029 seconds
Convert nissl images to gray...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1053.684169 seconds
16 nodes requested, 16 nodes available...Continuing


Match intensity profile between Neurotrace and Nissl...

Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
Convert neurotrace images to gray...16 nodes requested, 16 nodes available...Continuing


 done in 3785.25425816 seconds


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1298.919096 seconds


# Convert images to nissl-like grayscale - for full neurotrace stacks

In [9]:
for stack in ['MD635']:
    
    # Convert Neurotrace images to grayscale using a priori intensity mapping.
    
    t = time.time()
    sys.stderr.write('Convert neurotrace images to gray...')
    
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')
    
    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': metadata_cache['valid_filenames'][stack]},
                        argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2000 seconds.    

Convert neurotrace images to gray...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 1952.800308 seconds


# ONE STACK

# Step 1: Convert Nissl images to grayscale

In [ ]:
t = time.time()
sys.stderr.write('Convert nissl images to gray...')
               
script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\''%\
                    {'script_path': script_fp, 'stack': stack},
#                     kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                    kwargs_list={'filenames':
                                 [fn for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
                                  if fn.split('-')[1][0] == 'N']},
                    argument_type='list2',
                   cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1200 seconds.

In [44]:
# Convert contrast-stretched Neurotrace images to grayscale

# script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace.py')
# in_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped_contrast_stretched')
# out_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped_contrast_stretched_blueasgray')
# ! mkdir -p {out_dir}

In [57]:
# t = time.time()
# sys.stderr.write('Convert neurotrace images to gray...')
               
# run_distributed(command='%(script_path)s %%(in_fn)s %%(out_fn)s'%\
#                     {'script_path': script_fp},
#                     kwargs_list=[{'in_fn': os.path.join(in_dir, fn + '_lossless_alignedTo_' + anchor_fn + '_cropped_contrast_stretched.tif'),
#                                 'out_fn': os.path.join(out_dir, fn + '_lossless_alignedTo_' + anchor_fn + '_cropped_contrast_stretched_blueasgray.tif')}
# #                                 for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]
#                                  for fn in valid_filenames[150:151]
#                                 if fn.split('-')[1][0] == 'F'],
#                     argument_type='single',
#                    cluster_size=16)

# wait_qsub_complete()

# sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2500 seconds

Convert neurotrace images to gray...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 75.575123 seconds


# Step 2: Match intensity profile between Neurotrace Blue to Nissl

In [122]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD653', 'MD652', 'MD642', 'MD658']:

#     download_from_s3(DataManager.get_sorted_filenames_filename(stack=stack))
#     _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
#     valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
#     download_from_s3(DataManager.get_anchor_filename_filename(stack=stack))
#     anchor_fn = DataManager.load_anchor_filename(stack=stack)
#     download_from_s3(DataManager.get_cropbox_filename(stack=stack))
#     xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
#     w = xmax + 1 - xmin
#     h = ymax + 1 - ymin
#     x = xmin
#     y = ymin
#     first_fn = sections_to_filenames[first_sec]
#     last_fn = sections_to_filenames[last_sec]
#     first_idx_among_valid = valid_filenames.index(first_fn)
#     last_idx_among_valid = valid_filenames.index(last_fn)
    
    #########################################################
    
    t = time.time()
    print 'Match intensity profile between Neurotrace and Nissl...',
    
    filename_pairs = []
    l = metadata_cache['valid_filenames'][stack]
    for i, fn in enumerate(l):
        if l[i].split('-')[1][0] == 'F':
            for d in range(1, 99):
                if i+d < len(l) and l[i+d].split('-')[1][0] == 'N':
                    filename_pairs.append((l[i+d], l[i]))
                    break
                if i-d >= 0 and l[i-d].split('-')[1][0] == 'N':
                    filename_pairs.append((l[i-d], l[i]))
                    break
    
    script = os.path.join(REPO_DIR, 'preprocess', 'match_intensity_profile_v2.py')
    
    run_distributed(command='%(script_path)s %(stack)s \'%%(filename_pairs)s\' 100' % \
                    {'script_path': script,
                    'stack': stack},
#                     kwargs_list=dict(filename_pairs=filename_pairs[37:38]),
                    kwargs_list=dict(filename_pairs=filename_pairs),
                    argument_type='list2'
                   )

    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds'

Match intensity profile between Neurotrace and Nissl... rm -f ~/stderr_*; rm -f ~/stdout_*


Child returned 0
16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
Child returned 0
16 nodes available.


done in 2790.67252398 seconds
Match intensity profile between Neurotrace and Nissl... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
Child returned 0
16 nodes available.


done in 2554.21281099 seconds
Match intensity profile between Neurotrace and Nissl... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
Child returned 0
16 nodes available.


done in 2655.20303488 seconds
Match intensity profile between Neurotrace and Nissl... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 2363.96758008 seconds


qsub returned.


# Step 3: Convert Neurotrace images to grayscale (New, use nonlinear intensity mapping)

In [6]:
request_compute_nodes(16, 'yuncong')

Setting autoscaling group cfncluster-yuncongcluster-ComputeFleet-1LEH9CIEITL1A capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


In [7]:
run_distributed('rm -rf /scratch/*', argument_type='single')

rm -f ~/stderr_*; rm -f ~/stdout_*


0 nodes available.


In [8]:
# for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]:
#     download_from_s3(DataManager.get_image_filepath(stack=stack, version='cropped', resol='lossless', fn=fn))

This requires at least 4.2xlarge instances. 4.xlarge causes memory error.

In [101]:
# _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
# valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [8]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

In [9]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD652', 'MD653']:
    
#     output_dir = create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=2, resol='lossless'));
#     ! rm -r {output_dir}

    t = time.time()
    sys.stderr.write('Convert neurotrace images to gray...')

    run_distributed(command='DATA_DIR=/scratch/CSHL_data_processed %(script_path)s %(stack)s \'%%(filenames)s\' -o %(output_version)s --not_use_section_specific'%\
#     run_distributed(command='DATA_DIR=/scratch/CSHL_data_processed %(script_path)s %(stack)s \'%%(filenames)s\' -o %(output_version)s'%\
                        {'script_path': script_fp, 
                        'stack': stack,
                        'output_version': 'grayDefault'},
                        kwargs_list={'filenames': metadata_cache['valid_filenames'][stack]
#                                      [fn for fn in metadata_cache['valid_filenames'][stack][151:152]
#                                       if fn.split('-')[1][0] == 'F']
                                    },
                        argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1200 seconds.
    
# 16 4.2xlarge, whole stack, 2000s

Convert neurotrace images to gray...16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 1885.043394 seconds
Convert neurotrace images to gray...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 2247.007864 seconds


In [68]:
# upload_to_s3(DataManager.get_image_dir(stack=stack, version='cropped_gray', resol='lossless'), is_dir=True)

# Step 2/3: Convert Neurotrace images to grayscale (linear intensity mapping)

In [2]:
stack = 'MD662'

In [3]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

In [5]:
run_distributed('rm -rf /scratch/*', argument_type='single')
wait_qsub_complete()

rm -f ~/stderr_*; rm -f ~/stdout_*


0 nodes available.
qsub returned.


In [8]:
t = time.time()
sys.stderr.write('Convert neurotrace images to gray...')

for low in range(500, 1500, 500):
    high = 0
# for low in [500]:

    run_distributed(command='DATA_ROOTDIR=/scratch %(script_path)s %(stack)s \'%%(filenames)s\' -l %(low)d -H %(high)d -o %(output_version)s'%\
                        {'script_path': script_fp, 
                        'stack': stack,
                        'low': low,
                         'high': high,
                        'output_version': 'grayL'+str(low)+'H'+str(high)},
                        kwargs_list={'filenames': 
                                     [fn for fn in metadata_cache['valid_filenames'][stack]
                                      if fn.split('-')[1][0] == 'F']},
#                                      metadata_cache['valid_filenames'][stack]},
                        argument_type='list2',
                   node_list=get_node_list())

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# /scratch, 133 jobs / 16 nodes: 850 seconds

Convert neurotrace images to gray...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 672.223936 seconds
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 1264.206029 seconds


In [ ]:
# This may not be necessary. Intensity rescaling is easy at feature generation time.

script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')

for stack in ['MD585', 'MD589', 'MD594']:
    
    run_distributed('rm -rf /scratch/*', argument_type='single')
    wait_qsub_complete()

    for low in [0, 63, 127]:
        for high in [127, 191, 255]:
            if low < high:
                
                t = time.time()
                sys.stderr.write('Convert nissl images to gray...')

                run_distributed(command='DATA_DIR=/scratch/CSHL_data_processed %(script_path)s %(stack)s \'%%(filenames)s\' -l %(low)d -H %(high)d -o %(output_version)s'%\
                                    {'script_path': script_fp, 
                                    'stack': stack,
                                    'low': low,
                                     'high': high,
                                    'output_version': 'grayLinear'+str(low)+'t'+str(high)},
                                    kwargs_list={'filenames': metadata_cache['valid_filenames'][stack]},
                                    argument_type='list2',
                               node_list=get_node_list())

                wait_qsub_complete()

                sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

            # /scratch, 133 jobs / 16 nodes: 850 seconds

# Compress nissl gray and normalized fluorescent gray images (NEW)

In [12]:
run_distributed('rm -r /scratch/*', argument_type='single')

rm -f ~/stderr_*; rm -f ~/stdout_*


16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [2]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD642', 'MD652', 'MD653', 'MD657']:
    
    t = time.time()
    print 'Generating compressed images...',

    script_fp = os.path.join(REPO_DIR, 'preprocess', 'compress_as_jpeg.py')

    # Only specifying DATA_DIR does not work because download_from_s3() forms the path relative to ROOT_DIR.
    run_distributed('ROOT_DIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\"' % \
                    {'script': script_fp},
                    kwargs_list=[{'input_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
                                                                    version='grayDefault', data_dir='/scratch/CSHL_data_processed'),
                                  'output_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
                                                                    version='grayDefaultJpeg',data_dir='/scratch/CSHL_data_processed',ext='jpg')}
                                 for fn in metadata_cache['valid_filenames'][stack]],
                        argument_type='single')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # for one stack 300 seconds.

Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
16 nodes available.


done in 271.186305046 seconds
Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
16 nodes available.


done in 236.153964996 seconds
Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
16 nodes available.


done in 266.273519039 seconds
Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 261.257901907 seconds


qsub returned.


# Compress Nissl

In [21]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD585']:
    
    t = time.time()
    print 'Generating compressed images...',

    script_fp = os.path.join(REPO_DIR, 'preprocess', 'compress_as_jpeg.py')

    # Only specifying DATA_DIR does not work because download_from_s3() forms the path relative to ROOT_DIR.
#     run_distributed('ROOT_DIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\"' % \
#                     {'script': script_fp},
#                     kwargs_list=[{'input_fp': 
#                                   DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
#                                                                     version='Ntb', data_dir='/scratch/CSHL_data_processed'),
#                                   'output_fp': 
#                                   DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
#                                                                     version='NtbJpeg',data_dir='/scratch/CSHL_data_processed',ext='jpg')}
#                                  for fn in metadata_cache['valid_filenames'][stack]],
#                         argument_type='single',
#                    local_only=True)


    run_distributed('ROOT_DIR=/media/yuncong/BstemAtlasData/ %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\"' % \
                    {'script': script_fp},
                    kwargs_list=[{'input_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=3, fn=fn, resol='lossless', 
                                                                    data_dir=DATA_DIR),
                                  'output_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=3, fn=fn, resol='lossless', 
                                                                    version='jpeg', data_dir=DATA_DIR, ext='jpg')}
                                 for fn in metadata_cache['valid_filenames'][stack]],
                        argument_type='single',
                   local_only=True)


#     wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # for one stack 300 seconds.

Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
done in 1660.535950 seconds


In [20]:
! cat /tmp/stdout_0.log

upload: ../../../media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_lossless_jpeg/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_lossless_jpeg.jpg to s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_prep2_lossless_jpeg/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_lossless_jpeg.jpg
Setting environment for Precision WorkStation
convert "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep3_lossless/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep3_lossless.tif" -depth 8 -format jpg -quality 80 "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_lossless_jpeg/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_lossless_jpeg.jpg"
aws s3 cp "/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_lossless_jpeg/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_lossless_jpeg.jpg" "s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_prep2_lossless_jpeg/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_lossless_jpeg.jpg"


# Compress CryoJane data to JPEG

In [11]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['ChatCryoJane201710']:
    
    t = time.time()
    print 'Generating compressed images...',

    script_fp = os.path.join(REPO_DIR, 'preprocess', 'compress_as_jpeg.py')

    # Only specifying DATA_DIR does not work because download_from_s3() forms the path relative to ROOT_DIR.
#     run_distributed('ROOT_DIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\"' % \
#                     {'script': script_fp},
#                     kwargs_list=[{'input_fp': 
#                                   DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
#                                                                     version='Ntb', data_dir='/scratch/CSHL_data_processed'),
#                                   'output_fp': 
#                                   DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
#                                                                     version='NtbJpeg',data_dir='/scratch/CSHL_data_processed',ext='jpg')}
#                                  for fn in metadata_cache['valid_filenames'][stack]],
#                         argument_type='single',
#                    local_only=True)


    run_distributed('ROOT_DIR=/media/yuncong/BstemAtlasData/ %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\"' % \
                    {'script': script_fp},
                    kwargs_list=[{'input_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
                                                                    version='Chat', data_dir=DATA_DIR),
                                  'output_fp': 
                                  DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', 
                                                                    version='ChatJpeg', data_dir=DATA_DIR, ext='jpg')}
                                 for fn in metadata_cache['valid_filenames'][stack]],
                        argument_type='single',
                   local_only=True)


#     wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # for one stack 300 seconds.

[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
done in 4548.938718 seconds


# Shrink JPEG

In [3]:
downscale = 8

In [4]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'rescale_image.py')

In [7]:
# for stack in all_alt_nissl_ntb_stacks + all_alt_nissl_tracing_stacks:
for stack in ['MD652', 'MD653', 'MD657', 'MD658']:
    
    t = time.time()
    print 'Generating compressed images...',
    w, h = metadata_cache['image_shape'][stack]
    
    if downscale == 32:

        run_distributed('ROOT_DIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\" -W %(w)d -H %(h)d' % \
                {'script': script_fp, 'w': w/32, 'h': h/32},
                kwargs_list=[{'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='lossless', version='grayJpeg', data_dir='/scratch/CSHL_data_processed',ext='jpg'),
                              'output_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='thumbnail', version='grayJpeg', thumbnail_data_dir='/scratch/CSHL_data_processed',ext='jpg')}
                             for fn in metadata_cache['valid_filenames'][stack]],
                    argument_type='single')
        
    else:
        
        # Only specifying DATA_DIR does not work because download_from_s3() forms the path relative to ROOT_DIR.
        run_distributed('ROOT_DIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\" -W %(w)d -H %(h)d' % \
                        {'script': script_fp, 'w': w/downscale, 'h': h/downscale},
                        kwargs_list=[{'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn,\
                                                                                    resol='lossless', \
                                                                                    version='grayDefaultJpeg', data_dir='/scratch/CSHL_data_processed',ext='jpg'),
                                      'output_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, \
                                                                                     resol='down%d'%downscale, \
                                                                                     version='grayDefaultJpeg', data_dir='/scratch/CSHL_data_processed',ext='jpg')}
                                     for fn in metadata_cache['valid_filenames'][stack]],
                            argument_type='single')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 135.785959 seconds
16 nodes available.


Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 150.879550 seconds
16 nodes available.


Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 155.894895 seconds
16 nodes available.


Generating compressed images... rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 140.805168 seconds


# Compress and shrink.

In [2]:
downscale = 8

In [6]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'compress_and_rescale.py')

In [7]:
for (in_version, out_version) in [('gray', 'grayJpeg')]:
    for stack in all_nissl_stacks:

        t = time.time()
        print 'Compress and rescale images...',
        w, h = metadata_cache['image_shape'][stack]
        run_distributed('DATA_ROOTDIR=/scratch %(script)s \"%%(input_fp)s\" \"%%(output_fp)s\" -W %(w)d -H %(h)d' % \
                        {'script': script_fp, 'w': w/downscale, 'h': h/downscale},
                        kwargs_list=[{'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, 
                                                                                    resol='lossless', 
                                                                                    version=in_version, 
                                                                                    data_dir='/scratch/CSHL_data_processed'),
                                      'output_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, 
                                                                                     resol='down'+str(downscale), 
                                                                                     version=out_version,
                                                                                     data_dir='/scratch/CSHL_data_processed', ext='jpg')}
#                                      for fn in metadata_cache['valid_filenames'][stack] if fn.split('-')[1][0] == 'F'],
                                     for fn in metadata_cache['valid_filenames'][stack]],
                            argument_type='single')

        wait_qsub_complete()

        print 'done in', time.time() - t, 'seconds' 

    # 16 m4.4xlarge, one stack, 115 seconds.

Compress and rescale images... rm -f ~/stderr_*; rm -f ~/stdout_*


5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 321.014811039 seconds
Compress and rescale images... rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 280.957913876 seconds
Compress and rescale images... rm -f ~/stderr_*; rm -f ~/stdout_*


...

qsub returned.
5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 321.06528306 seconds
Compress and rescale images... rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


done in 331.033967972 seconds
Compress and rescale images... rm -f ~/stderr_*; rm -f ~/stdout_*
done in 376.182645082 seconds


qsub returned.


# Constrast stretch 16-bit fluorescent image, generate RGB JPEG (NEW)

In [2]:
run_distributed('rm -r /scratch/*', argument_type='single')

rm -f ~/stderr_*; rm -f ~/stdout_*


0 nodes available.


In [10]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'contrast_stretch_fluorescent_image.py')

In [10]:
# on_s3 = [os.path.splitext(r.split()[3])[0][:-len('_prep2_lossless_contrastStretched')]
#  for r in check_output("aws s3 ls s3://mousebrainatlas-data/CSHL_data_processed/MD661/MD661_prep2_lossless_contrastStretched/".split()).split('\n')
#  if len(r.split()) > 1]
# print set(on_s3) - set(metadata_cache['valid_filenames'][stack])
# print set(metadata_cache['valid_filenames'][stack]) - set(on_s3)

In [11]:
# import psutil
# vmem = psutil.virtual_memory()
# print vmem.total

In [28]:
for stack in ['MD658']:

    t = time.time()
    
    if stack == 'MD658':
        label_channel = 0
        label_clipmax = 1300
    else:
        label_channel = 1
        label_clipmax = 400
    

    run_distributed(command='DATA_ROOTDIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\' -c %(label_channel)d -max %(label_clipmax)d'%\
                        {'script_path': script_fp, 
                        'stack': stack,
                        'label_channel': label_channel,
                        'label_clipmax': label_clipmax},
                        kwargs_list={'filenames': 
                                     [fn for fn in metadata_cache['valid_filenames'][stack]
                                      if fn.split('-')[1][0] == 'F']
#                                      ['MD658-F33-2017.04.04-23.12.27_MD658_1_0097']
                                    },
                        argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

rm -f ~/stderr_*; rm -f ~/stdout_*


16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 522.101666 seconds


# Generate Masks for Aligned Sections (new)

- Download aligned thumbnails to local machine.
- Run `mask_editing_gui.py`. Draw initial contours. Upload `initSnakeContours.pkl` to S3.

In [4]:
# download_from_s3(DataManager.get_image_dir_v2(stack=stack, prep_id=1, resol='thumbnail'), is_dir=True)
download_from_s3(DataManager.get_image_dir_v2(stack=stack, prep_id=1, resol='thumbnail', version='Ntb'), is_dir=True)

rm -rf "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_Ntb" && mkdir -p "/shared/CSHL_data_processed/ChatCryoJane201710"
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_Ntb" "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_Ntb"


In [4]:
script = os.path.join(REPO_DIR, 'preprocess', 'generate_thumbnail_masks_v5.py')

output_dir = create_if_not_exists(DataManager.get_auto_submask_rootdir_filepath(stack=stack))
! rm -rf {output_dir} # important to clear remaining folders.

In [5]:
init_snake_contours_fp = DataManager.get_initial_snake_contours_filepath(stack=stack)
download_from_s3(init_snake_contours_fp, redownload=True)

rm -rf "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_initSnakeContours.pkl" && mkdir -p "/shared/CSHL_data_processed/ChatCryoJane201710"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_initSnakeContours.pkl" "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_initSnakeContours.pkl"


In [60]:
# _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
# valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [6]:
t = time.time()
print 'Generating thumbnail mask...',

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(init_snake_contours_fp)s --min_size 500 --default_channel 1 --shrink 3 --version Ntb' % \
                {'script_path': script,
                'stack': stack,
                'init_snake_contours_fp': init_snake_contours_fp},
                kwargs_list={'filenames': metadata_cache['valid_filenames_all'][stack]},
                argument_type='list2',
               local_only=True)

# wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 300s (aws)

Generating thumbnail mask... rm -f ~/stderr_*; rm -f ~/stdout_*


Run locally.


done in 1494.530756 seconds


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


- download the folder `autoSubmasks` from S3.
- Use local mask editing tool, generate `userModifiedSubmasks` and `masks`, upload to S3.

# Generate Masks

In [11]:
script = os.path.join(REPO_DIR, 'preprocess', 'generate_thumbnail_masks_v4.py')
input_dir = os.path.join(RAW_DATA_DIR, stack)
output_dir = create_if_not_exists(os.path.join(DATA_DIR, stack, stack + '_submasks'))
! rm -f output_dir/*

In [16]:
t = time.time()
print 'Generating thumbnail mask...',

wait_num_nodes(16)

run_distributed(command='%(script_path)s %(stack)s %(input_dir)s \'%%(filenames)s\' %(output_dir)s --border_dissim_percentile %(border_dissim_percentile)d --min_size %(min_size)d' % \
                {'script_path': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir,
                'border_dissim_percentile': DEFAULT_BORDER_DISSIMILARITY_PERCENTILE,
                'min_size': DEFAULT_MINSIZE},
                kwargs_list=dict(filenames=valid_filenames),
                exclude_nodes=[33],
                argument_type='list2',
               cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 300s (aws)

Generating thumbnail mask... Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1LFRACYHLTNL8 capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


Wait for SGE to know all nodes (timeout in 300 seconds)...
All nodes are ready.
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


done in 403.646880865 seconds


qsub returned.


In [18]:
transfer_data_synced(relative_to_ec2(output_dir),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive /shared/CSHL_data_processed/MD594/MD594_submasks s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_submasks


Child returned 0
3.72 seconds.


- download `submasks/` to local machine
- review them in GUI
- generate `submasks_modified/`, `masks/`, `submasks_finalDecisions.txt`, upload to S3

# Warp Thumbnail Masks

In [6]:
script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v3.py')
input_dir = os.path.join(DATA_DIR, stack, stack + '_masks')

In [31]:
output_dir = os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn)

execute_command('rm -rf ' + output_dir)

transforms_to_anchor = load_pickle(DataManager.get_transforms_filename(stack=stack))

rm -rf /shared/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_masks


Child returned 0


In [24]:
t = time.time()
print 'warping thumbnail mask...',

run_distributed('%(script)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 black' % \
                {'script': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir},
                kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                            'filename': fn + '_mask.png',
                            'output_fn': fn + '_mask_alignedTo_' + anchor_fn + '.png'}
                            for fn in valid_filenames],
                argument_type='single',
               cluster_size=16,
               jobs_per_node=16)

wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 20 seconds (aws)

warping thumbnail mask... Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1LFRACYHLTNL8 capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 1 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


done in 76.2961359024 seconds


qsub returned.


In [27]:
transfer_data_synced(relative_to_ec2(output_dir),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive /shared/CSHL_data_processed/MD594/MD594_masks_alignedTo_MD594-IHC58-2015.08.26-18.48.50_MD594_1_0172 s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_masks_alignedTo_MD594-IHC58-2015.08.26-18.48.50_MD594_1_0172


Child returned 0
3.42 seconds.


# Crop Thumbnail Masks (new)

In [220]:
# stack = 'MD661'

# download_from_s3(DataManager.get_sorted_filenames_filename(stack=stack))
# _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
# valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
# download_from_s3(DataManager.get_anchor_filename_filename(stack=stack))
# anchor_fn = DataManager.load_anchor_filename(stack=stack)
# download_from_s3(DataManager.get_cropbox_filename(stack=stack))
# xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
# w = xmax + 1 - xmin
# h = ymax + 1 - ymin
# x = xmin
# y = ymin
# first_fn = sections_to_filenames[first_sec]
# last_fn = sections_to_filenames[last_sec]
# first_idx_among_valid = valid_filenames.index(first_fn)
# last_idx_among_valid = valid_filenames.index(last_fn)

In [32]:
####

In [10]:
# stack = 'MD661'
stack = 'ChatCryoJane201710'
xmin, xmax, ymin, ymax, _, _ = DataManager.load_cropbox(stack=stack)
print xmin, xmax, ymin, ymax

509 1457 72 767


In [11]:
input_dir = DataManager.get_thumbnail_mask_dir_v3(stack=stack, prep_id=1)
download_from_s3(input_dir, is_dir=True)

output_dir = DataManager.get_thumbnail_mask_dir_v3(stack=stack, prep_id=2)

execute_command('rm -rf ' + output_dir);
execute_command('mkdir -p ' + output_dir);

rm -rf "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask" && mkdir -p "/shared/CSHL_data_processed/ChatCryoJane201710"
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask" "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask"
rm -rf /shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask
mkdir -p /shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask


In [34]:
# _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack) 
# valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [12]:
t = time.time()
sys.stderr.write('cropping thumbnail mask...')

# for fn in valid_filenames:
for fn in metadata_cache['valid_filenames'][stack]:    
    
    execute_command('convert \"%(input_fp)s\" -crop %(w)dx%(h)d+%(x)d+%(y)d \"%(output_fp)s\"' % \
        {'stack': stack,
        'input_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=1, fn=fn, resol='thumbnail', version='mask', ext='png'),
        'output_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=2, fn=fn, resol='thumbnail', version='mask', ext='png'),
        'w':xmax+1-xmin, 'h':ymax+1-ymin, 'x':xmin, 'y':ymin})

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70s (aws)

cropping thumbnail mask...

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide1-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide1-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide2-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide2-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide1-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide1-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide10-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide10-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide9-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide9-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide10-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide10-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane20

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide19-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide19-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide20-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide20-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide19-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide19-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide28-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide28-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide27-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide27-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide28-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide28-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide37-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide37-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide38-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide38-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide37-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide37-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide46-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide46-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide45-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide45-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide46-S3_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide46-S3_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane

convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide55-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide55-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide56-S1_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide56-S1_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep1_thumbnail_mask/ChatCryoJane201710_slide55-S2_prep1_thumbnail_mask.png" -crop 949x696+509+72 "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask/ChatCryoJane201710_slide55-S2_prep2_thumbnail_mask.png"
convert "/shared/CSHL_data_processed/ChatCryoJane

done in 12.888359 seconds


In [121]:
# t = time.time()
# sys.stderr.write('cropping thumbnail mask...')

# execute_command('mogrify -set filename:name %%t -crop %(w)dx%(h)d+%(x)d+%(y)d -write \"%(output_dir)s/%%[filename:name]_cropped.png\" \"%(input_dir)s/*.png\"' % \
#     {'stack': stack,
#     'input_dir': input_dir,
#     'output_dir': output_dir,
#     'w':xmax+1-xmin, 'h':ymax+1-ymin, 'x':xmin, 'y':ymin})

# sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70s (aws)

In [13]:
upload_to_s3(output_dir, is_dir=True)

aws s3 cp --recursive "/shared/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask" "s3://mousebrainatlas-data/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_mask"


- Run `extract_test_features_cnn.ipynb` on workstation.
- Upload to extracted features to S3.
- Continue with `learning/pipeline_aws.ipynb`